Курсовой проект по курсу NLP Никифоренко Дмитрий
* задачи изменил по ходу работы по причине недостаточного понимания материала, ограниченности сроков сдачи и большой комплексности курсового проекта
* был сформирован чат-бот в телеграм умеющий болтать на свободную тему, предлагать товар интернет магазина по запросу, предлагать лекарственные препараты по запросу фармакологической группы и формы выпуска лекарства, рассказывать сказку/историю по запросу путём генерации текста на базе GPT2_small от сбера.
* в данном блокноте отражены этапы подготовки и обучения доменов товаров магазина, лекарственных препаратов и свободного общения.
* этапы подготовки модели генерации текста отражены в блокноте tailes.ipynb
* код работы чат бота находится в блокноте Telegram_bot.ipynb
* все дампы, необходимые для работы моделей находятся в двух каталогах на гугл диске с общим доступом. Vectorizer, tfidf, таблицы индексов ответов и векторов для болталки, магазина товаров и лекарственных препаратов расположены в каталоге https://drive.google.com/drive/folders/1XMPvhwBYpKddYAE2iISXFbuWN-TFSOCl?usp=drive_link Всё необходимое для загрузки модели генерации текста для сказок/историй находится в катологе https://drive.google.com/drive/folders/1Dc0DTR30QP-13c92lUrtiUS5w-CNoMMu?usp=drive_link

Установка основных зависимостей и библиотек

In [ ]:
! pip install python-telegram-bot==13.3 pymorphy2 stop_words annoy num2words

In [ ]:
import os
from telegram.ext import Updater, CommandHandler, MessageHandler, filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import json
import num2words

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#количество сторок в объекте
import mmap
import re

def get_num_lines(file_path):
  fp = open(file_path, 'r+')
  buf = mmap.mmap(fp.fileno(), 0)
  lines = 0
  while buf.readline():
    lines += 1
  return lines

Работа над вопросно-ответной системой

In [ ]:
# Преобразование файла аопросов-ответов в строчный вид
if not os.path.isfile('/content/drive/MyDrive/chat_bot_telegram/prepared_answers.txt'):

    question = None
    written = False

    with open("/content/drive/MyDrive/chat_bot_telegram/prepared_answers.txt", "w") as fout:
        with open("/content/drive/MyDrive/chat_bot_telegram/Otvety.txt", "r") as fin:
            for line in tqdm(fin, total=get_num_lines("/content/drive/MyDrive/chat_bot_telegram/Otvety.txt")):
                if line.startswith("---"):
                    written = False
                    continue
                if not written and question is not None:
                    fout.write(question.replace("\t", " ").strip() + "===" + line.replace("\t", " "))
                    written = True
                    question = None
                    continue
                if not written:
                    question = line.strip()
                    continue

In [ ]:
# Обработка текста

sentences = []
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))

In [ ]:
def preprocess_txt(line):
    spls = ' '.join([re.sub(r'={3}', ' === ', i) for i in line.split()]).split()
    spls = [i for i in spls if i not in sw and i != ""]
    spls = [re.sub(r'[^а-яА-Яё=== ]', '', i) for i in spls]
    spls = [i for i in spls if len(i)>2]
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    return spls

In [ ]:
file_path_from = '/content/drive/MyDrive/chat_bot_telegram/prepared_answers.txt'
file_path_to = '/content/drive/MyDrive/chat_bot_telegram/prepared_answers2.txt'

In [ ]:
if not os.path.isfile(file_path_to):

    N = get_num_lines(file_path_from)
    with open(file_path_to, mode = 'w') as fileto:
        with open(file_path_from) as filefrom:
            for k in tqdm(range(N)):
                line = filefrom.readline()
                if line == '': break
                spls = preprocess_txt(line)
                sentences.append(spls)

                fileto.write(' '.join(spls)+'\n')
    filefrom.close()
    fileto.close()

In [ ]:
# Загрузить результат

sentences = []

file_path_from = '/content/drive/MyDrive/chat_bot_telegram/prepared_answers2.txt'
if os.path.isfile(file_path_from):
    N = get_num_lines(file_path_from)
    with open(file_path_to, mode = 'r') as filefrom:
        for k in tqdm(range(N)):
            line = filefrom.readline()
            if line == '': break
            sentences.append(line.split())
    filefrom.close()

In [ ]:
# Обучим модель FastText

file_path_from = '/content/drive/MyDrive/chat_bot_telegram/ft_model'
if not os.path.isfile(file_path_from):

    sentences = [i for i in tqdm(sentences) if len(i) > 2]
    modelFT = FastText(sentences=sentences, vector_size=100, min_count=1, window=5)
    modelFT.save("/content/drive/MyDrive/chat_bot_telegram/ft_model")
    ft_index = annoy.AnnoyIndex(100 ,'angular')

In [ ]:
#загрузка FastText
modelFT = FastText.load("/content/drive/MyDrive/chat_bot_telegram/ft_model")

In [ ]:
ft_index = annoy.AnnoyIndex(100 ,'angular')

In [ ]:
# Создаем Индексы для вопросов-ответов

file_path_from = '/content/drive/MyDrive/chat_bot_telegram/speaker.ann'
if not os.path.isfile(file_path_from):

    index_map = {}
    counter = 0
    with open('/content/drive/MyDrive/chat_bot_telegram/prepared_answers.txt', "r") as f:
        for line in tqdm(f, total=get_num_lines('/content/drive/MyDrive/chat_bot_telegram/prepared_answers.txt')):
            n_ft = 0
            spls = line.split("===")
            index_map[counter] = spls[1]
            question = preprocess_txt(spls[0])
            question = [i for i in question if len(i)>2]
            vector_ft = np.zeros(100)
            for word in question:
              if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
            if n_ft > 0:
                vector_ft = vector_ft / n_ft
            ft_index.add_item(counter, vector_ft)

            counter += 1

    ft_index.build(10)
    ft_index.save('/content/drive/MyDrive/chat_bot_telegram/speaker.ann')

    with open("/content/drive/MyDrive/chat_bot_telegram/index_speaker.pkl", "wb") as f:
        pickle.dump(index_map, f)

  0%|          | 0/1163342 [00:00<?, ?it/s]

Product Task - товары магазина

In [ ]:
shop_data = pd.read_csv("/content/drive/MyDrive/chat_bot_telegram/ProductsDataset.csv")


shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, новый, носить, раз, реал, крас..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, брюки, новый, знать, мерило, п..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, размерсостояние, отличное..."


In [ ]:
shop_data.to_csv('/content/drive/MyDrive/chat_bot_telegram/shop_data.csv')

In [ ]:
shop_data = pd.read_csv('/content/drive/MyDrive/chat_bot_telegram/shop_data.csv')

Лекарственные препараты

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/chat_bot_telegram/datadrugsrosreestr032019.csv')

In [ ]:
df.drop(df[df['pharmacotherapeuticgroup'].isna()].index, axis=0, inplace=True)
df.drop(df[df['country'].isna()].index, axis=0, inplace=True)
df.drop(df[df['formrelease'].isna()].index, axis=0, inplace=True)
df.drop(df[df['pharmacotherapeuticgroup'] == '~'].index, axis=0, inplace=True)
df.drop(df[df['normativedocumentation'].isna()].index, axis=0, inplace=True)
df.drop(['regnumber',
         'regdate',
         'enddate',
         'cancellationdate',
         'internationalname',
         'stages',
         'barcodes',
         'tradename'], axis=1, inplace=True)

In [ ]:
df = pd.concat([df, df[:15000]])

In [ ]:
df['text'] = df['formrelease'] + " " + df["pharmacotherapeuticgroup"]
df['text'] = df['text'].apply(lambda x: preprocess_txt(str(x)))

In [ ]:
df.to_csv('/content/drive/MyDrive/chat_bot_telegram/pharm.csv')

In [ ]:
# Подготовка для создания модели для определения домена данных

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [ ]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
# Вопрос-ответный домен
cls_0 = [" ".join(preprocess_txt(index_map[i])) for i in tqdm(idxs, total=len(idxs))]
# Продуктовый домен
cls_1 = [" ".join(val) for val in tqdm(shop_data['text'].values, total=len(shop_data))]

  0%|          | 0/35042 [00:00<?, ?it/s]

  0%|          | 0/35548 [00:00<?, ?it/s]

In [ ]:
pickle.dump(cls_0, open('/content/drive/MyDrive/chat_bot_telegram/cls_0.pkl', 'wb'))
pickle.dump(cls_1, open('/content/drive/MyDrive/chat_bot_telegram/cls_1.pkl', 'wb'))

In [ ]:
#Примеры запросов на Story Tale
cnt = 0
cls_2 = []
while cnt != 35000:
  cls_2.append(' '.join(preprocess_txt(quest[np.random.randint(0, len(quest))])))
  cnt += 1

In [ ]:
pickle.dump(cls_2, open('/content/drive/MyDrive/chat_bot_telegram/cls_2.pkl', 'wb'))

In [ ]:
cls_3 = [" ".join(val) for val in tqdm(df['text'].values, total=len(df))]

  0%|          | 0/34207 [00:00<?, ?it/s]

In [ ]:
pickle.dump(cls_3, open('/content/drive/MyDrive/chat_bot_telegram/cls_3.pkl', 'wb'))

In [ ]:
cls_0 = pickle.load(open('/content/drive/MyDrive/chat_bot_telegram/cls_0.pkl', 'rb'))
cls_1 = pickle.load(open('/content/drive/MyDrive/chat_bot_telegram/cls_1.pkl', 'rb'))
cls_2 = pickle.load(open('/content/drive/MyDrive/chat_bot_telegram/cls_2.pkl', 'rb'))
cls_3 = pickle.load(open('/content/drive/MyDrive/chat_bot_telegram/cls_3.pkl', 'rb'))

In [ ]:
# Формирование датасета: cls_0 - QA; cls_1 - товары магазина; cls_2 - Story Tale; cls_3 - лекарственные препараты

dataset = cls_0 + cls_1 + cls_2 + cls_3
dataset = pd.DataFrame(dataset, columns=['text'])

In [ ]:
#Labels
dataset['labels'] = 0
dataset.loc[len(cls_0)+1:len(cls_0)+len(cls_1), 'labels'] = 1
dataset.loc[len(cls_0)+len(cls_1)+1:len(cls_0)+len(cls_1)+len(cls_2), 'labels'] = 2
dataset.loc[len(cls_0)+len(cls_1)+len(cls_2)+1:, 'labels'] = 3

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset['text'], dataset['labels'], test_size=0.2,
                                                    stratify=dataset['labels'], random_state=13)

In [ ]:
# Модель

x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

In [ ]:
#save vect
pickle.dump(vectorizer.vocabulary_,open("/content/drive/MyDrive/chat_bot_telegram/vectorizer.vocabulary.pkl","wb"))

In [ ]:
lr = LogisticRegression(max_iter=150000).fit(x_train_vec, y_train)

In [ ]:
# сохранение линейной модели на диск
filename = '/content/drive/MyDrive/chat_bot_telegram/lr_model.sav'
pickle.dump(lr, open(filename, 'wb'))

In [ ]:
# Качество

from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9771816881258941

In [ ]:
# Добавим IDF взвешивание (для каждого слова найдем IDF вес)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [ ]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}

In [ ]:
with open('/content/drive/MyDrive/chat_bot_telegram/idfs.pkl', 'wb') as f:
    pickle.dump(idfs, f)

In [ ]:
# Создаем Индексы для продуктовых данных

file_path_from = '/content/drive/MyDrive/chat_bot_telegram/shop.ann'
if not os.path.isfile(file_path_from):


    ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

    midf = np.mean(tfidf_vect.idf_)

    index_map_shop = {}
    counter = 0

    for i in tqdm(range(len(shop_data))):
        n_ft = 0
        index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
        vector_ft = np.zeros(100)
        pre_txt = [i for i in shop_data.loc[i, "text"] if len(i)>2]

        for word in pre_txt:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word] * idfs.get(word, midf)
                n_ft += idfs.get(word, midf)
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index_shop.add_item(counter, vector_ft)
        counter += 1

    ft_index_shop.build(10)
    ft_index_shop.save('/content/drive/MyDrive/chat_bot_telegram/shop.ann')

    file_path_from = '/content/drive/MyDrive/chat_bot_telegram/index_shop.pkl'


    with open("/content/drive/MyDrive/chat_bot_telegram/index_shop.pkl", "wb") as f:
        pickle.dump(index_map_shop, f)


  0%|          | 0/35548 [00:00<?, ?it/s]

In [ ]:
# Создаем Индексы для базы данных медицинских препаратов

file_path_from = '/content/drive/MyDrive/chat_bot_telegram/pharm.ann'
if not os.path.isfile(file_path_from):


    ft_index_pharm = annoy.AnnoyIndex(100 ,'angular')

    midf = np.mean(tfidf_vect.idf_)

    index_map_pharm = {}
    counter = 0

    for i in tqdm(df.index):
        n_ft = 0
        index_map_pharm[counter] = (df.loc[i, "nameregcertificate"], df.loc[i, "country"], df.loc[i, "normativedocumentation"])
        vector_ft = np.zeros(100)
        pre_txt = [i for i in df.loc[i, "text"] if len(i)>2]

        for word in pre_txt:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word] * idfs.get(word, midf)
                n_ft += idfs.get(word, midf)
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index_pharm.add_item(counter, vector_ft)
        counter += 1

    ft_index_pharm.build(10)
    ft_index_pharm.save('/content/drive/MyDrive/chat_bot_telegram/pharm.ann')

    file_path_from = '/content/drive/MyDrive/chat_bot_telegram/index_pharm.pkl'

    with open("/content/drive/MyDrive/chat_bot_telegram/index_pharm.pkl", "wb") as f:
        pickle.dump(index_map_pharm, f)


  0%|          | 0/34207 [00:00<?, ?it/s]

In [ ]:
# Основная функция преобразования текста в вектор х100

def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft